In [67]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from nltk.corpus import stopwords
import re
cachedStopWords = stopwords.words("english")
columns=['label','document']

def replace_special_character(document):
    result = re.sub('[^a-zA-Z\n\.]', ' ', document).replace(".", "")
    result = ' '.join(result.split())
    result = "".join(result.splitlines())
    result=re.sub(r'\b\w{1,3}\b', '', result)
    return result.strip()

def removestopword(document):
    text = ' '.join([word for word in document.lower().split() if word not in cachedStopWords])
    return text


docIdLable = {}
def readdocument():
    data=[]
    files = ["data/train.dat"]
    for f in files:
        with open(f) as fl:
            idx = 0
            for lines in fl:
                temp=[lines[:2],lines[3:].strip(" ")]
                data.append(temp)
                docIdLable[idx] = lines[:2]
                idx +=1
    df=pd.DataFrame(data,columns=columns)
    df['document'] = df['document'].apply(lambda x: removestopword(x))
    df['document'] = df['document'].apply(lambda x: replace_special_character(x))
    vectorizer = CountVectorizer(min_df=1, max_features=2000)
    X = vectorizer.fit_transform(df['document'].values)
    transformer = TfidfTransformer(smooth_idf=False)
    tfidf=transformer.fit_transform(X)
    return tfidf

def readTestdocument(location):
    data=[]
    with open(location) as fl:
        for lines in fl:
            temp=lines.strip(" ")
            data.append(temp)
    df=pd.DataFrame(data,columns=['document'])
    df['document'] = df['document'].apply(lambda x: removestopword(x))
    df['document'] = df['document'].apply(lambda x: replace_special_character(x))
    vectorizer = CountVectorizer(min_df=1, max_features=2000)
    X = vectorizer.fit_transform(df['document'].values)
    transformer = TfidfTransformer(smooth_idf=False)
    tfidf=transformer.fit_transform(X)
    return tfidf

In [70]:


import numpy as np
from scipy import spatial
def cosineSimilarity(d1,d2, id):
    return 1 - spatial.distance.cosine(d1,d2)

# function to calculate K closest neighbours
import operator
import itertools
def kNearestNeighbours(trainVector, testVector, k):
    allDistances = {}
    for t in range(len(trainVector)):
        dist = spatial.distance.cosine(trainVector[t],testVector)
        allDistances[t] = dist
    return [(k,allDistances[k]) for k in sorted(allDistances, key=lambda x:allDistances[x],reverse=True)][:k]

# function to get predicted label based on the closest neighbours
import random
def getPredictedLabel(nearDocs):
    pos = 0
    neg = 0
    dict = {k:v for k,v in nearDocs} 
    for k in dict:
        if docIdLable[k] == '+1':
            pos+=1
        else:
            neg+=1
    
    genLab = None
    if(pos==neg):
        genLab =  random.sample(set([-1,1],1))
    else: 
        genLab = '+1' if pos> neg else '-1'
    print("calculated Label : ", genLab)
    return genLab

In [71]:
finalTrainTf = readdocument()
print("train doc read completed")
finalTestTf = readTestdocument("data/test.dat")
print("test doc read completed")
finalTrainTfArr = finalTrainTf.toarray()
finalTestTfArr = finalTestTf.toarray()


genLabels = []
k = 5
f = open('genLabels','w')
for tes in finalTestTfArr:
    nearest = kNearestNeighbours(finalTrainTfArr,tes,k)
    lab = getPredictedLabel(nearest)
    f.write(lab+"\n")
    genLabels.append(lab)
f.close()

train doc read completed


test doc read completed


calculated Label :  +1


NameError: name 'i' is not defined